In [1]:
%load_ext sparkmagic.magics

In [2]:
import os
from IPython import get_ipython
username = os.environ['RENKU_USERNAME']
server = "http://iccluster029.iccluster.epfl.ch:8998"

# set the application name as "<your_gaspar_id>-homework3"
get_ipython().run_cell_magic(
    'spark',
    line='config', 
    cell="""{{ "name": "{0}-homework3", "executorMemory": "4G", "executorCores": 4, "numExecutors": 10, "driverMemory": "4G"}}""".format(username)
)

In [3]:
get_ipython().run_line_magic(
    "spark", "add -s {0}-homework3 -l python -u {1} -k".format(username, server)
)

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
5983,application_1652960972356_1310,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


In [4]:
%%spark
print('We are using Spark %s' % spark.version)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

We are using Spark 2.3.2.3.1.4.0-315

In [5]:
%%spark?

Docstring:
::

  %spark [-c CONTEXT] [-s SESSION] [-o OUTPUT] [-q [QUIET]]
             [-m SAMPLEMETHOD] [-n MAXROWS] [-r SAMPLEFRACTION] [-u URL]
             [-a USER] [-p PASSWORD] [-t AUTH] [-l LANGUAGE] [-k [SKIP]]
             [-i ID] [-e COERCE]
             [command ...]

Magic to execute spark remotely.

This magic allows you to create a Livy Scala or Python session against a Livy endpoint. Every session can
be used to execute either Spark code or SparkSQL code by executing against the SQL context in the session.
When the SQL context is used, the result will be a Pandas dataframe of a sample of the results.

If invoked with no subcommand, the cell will be executed against the specified session.

Subcommands
-----------
info
    Display the available Livy sessions and other configurations for sessions.
add
    Add a Livy session given a session name (-s), language (-l), and endpoint credentials.
    The -k argument, if present, will skip adding this session if it already exist

### Import data

In [6]:
%%spark
all_stops = spark.read.orc('/data/sbb/orc/allstops')
all_stops.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- stop_id: string (nullable = true)
 |-- stop_name: string (nullable = true)
 |-- stop_lat: double (nullable = true)
 |-- stop_lon: double (nullable = true)
 |-- location_type: string (nullable = true)
 |-- parent_station: string (nullable = true)

In [7]:
%%spark
actual_data = spark.read.orc('/data/sbb/orc/istdaten')
actual_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- betriebstag: string (nullable = true)
 |-- fahrt_bezeichner: string (nullable = true)
 |-- betreiber_id: string (nullable = true)
 |-- betreiber_abk: string (nullable = true)
 |-- betreiber_name: string (nullable = true)
 |-- produkt_id: string (nullable = true)
 |-- linien_id: string (nullable = true)
 |-- linien_text: string (nullable = true)
 |-- umlauf_id: string (nullable = true)
 |-- verkehrsmittel_text: string (nullable = true)
 |-- zusatzfahrt_tf: string (nullable = true)
 |-- faellt_aus_tf: string (nullable = true)
 |-- bpuic: string (nullable = true)
 |-- haltestellen_name: string (nullable = true)
 |-- ankunftszeit: string (nullable = true)
 |-- an_prognose: string (nullable = true)
 |-- an_prognose_status: string (nullable = true)
 |-- abfahrtszeit: string (nullable = true)
 |-- ab_prognose: string (nullable = true)
 |-- ab_prognose_status: string (nullable = true)
 |-- durchfahrt_tf: string (nullable = true)

### Sub-sample

Sub-sample data for testing.

In [8]:
%%spark

# actual_data = actual_data.sample(0.1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Rename columns

Columns are renamed in English as follows:
- `betriebstag` to DATE
- `produkt_id` to PRODUCT_ID
- `fahrt_bezeichner` to TRIP_ID
- `bpuic` to STOP_ID
- `haltestellen_name` to STOP_NAME
- `ankunftszeit` to ARRIVAL_TIME
- `an_prognose_status` to ARRIVAL_TIME_STATUS
- `an_prognose` to ACTUAL_ARRIVAL_TIME
- `abfahrtszeit` to DEPARTURE_TIME
- `ab_prognose_status` to DEPARTURE_TIME_STATUS
- `ab_prognose` to ACTUAL_DEPARTURE_TIME

In [9]:
%%spark

#rename columns in english

actual_data = actual_data.select(actual_data.betriebstag.alias('DATE'),\
                                 actual_data.produkt_id.alias('PRODUCT_ID'),\
                                 actual_data.bpuic.alias('STOP_ID'),\
                                 actual_data.haltestellen_name.alias('STOP_NAME'),\
                                 actual_data.fahrt_bezeichner.alias('TRIP_ID'),\
                                 actual_data.ankunftszeit.alias('ARRIVAL_TIME'),\
                                 actual_data.an_prognose_status.alias('ARRIVAL_TIME_STATUS'),\
                                 actual_data.an_prognose.alias('ACTUAL_ARRIVAL_TIME'),\
                                 actual_data.abfahrtszeit.alias('DEPARTURE_TIME'),\
                                 actual_data.ab_prognose_status.alias('DEPARTURE_TIME_STATUS'),
                                 actual_data.ab_prognose.alias('ACTUAL_DEPARTURE_TIME'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Prepare table

In [10]:
%%spark
import pyspark.sql.functions as F
# F.date_format(F.to_timestamp(df_used.timestamp_s), 'yyyy-MM-dd HH:mm:ss').alias('date'))
df = actual_data.withColumn('DATE', F.date_format(F.to_timestamp(actual_data.DATE, 'dd.MM.yyyy'), 'yyyy-MM-dd'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
%%spark
import pyspark.sql.functions as F

df = df.withColumn('DAY_OF_WEEK', ((F.dayofweek(df['DATE'])+5)%7)+1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
%%spark
import pyspark.sql.functions as F

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS unknown
df = df.filter(df['ARRIVAL_TIME_STATUS'] != 'UNBEKANNT')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != 'UNBEKANNT')

#remove entries with ARRIVAL_TIME_STATUS and DEPARTURE_TIME_STATUS is empty
df = df.filter(df['ARRIVAL_TIME_STATUS'] != '')
df = df.filter(df['DEPARTURE_TIME_STATUS'] != '')

#remove entries with PRODUCT_ID empty
df = df.filter(df['PRODUCT_ID'] != '')

#remove entries with STOP_ID empty
df = df.filter(df['STOP_ID'] != '')

#remove entries with STOP_NAME empty
df = df.filter(df['STOP_NAME'] != '')

#remove entries with TRIP_ID empty
df = df.filter(df['TRIP_ID'] != '')

#transform to unix_timestamp
df = df.withColumn('ARRIVAL_TIME', F.unix_timestamp(df.ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_ARRIVAL_TIME', F.unix_timestamp(df.ACTUAL_ARRIVAL_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('DEPARTURE_TIME', F.unix_timestamp(df.DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))
df = df.withColumn('ACTUAL_DEPARTURE_TIME', F.unix_timestamp(df.ACTUAL_DEPARTURE_TIME, 'dd.MM.yyyy HH:mm'))

#fill null with 0 
#When the stop is the start or end of a journey, the corresponding columns will be empty (ANKUNFTSZEIT/ABFAHRTSZEIT)
df = df.na.fill({'ARRIVAL_TIME' : 0, 'ACTUAL_ARRIVAL_TIME' : 0, 'DEPARTURE_TIME' : 0, 'ACTUAL_DEPARTURE_TIME' : 0})

#compute delay
df = df.withColumn('ARRIVAL_DELAY', F.when((df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME'] )< 0, 0).otherwise(df['ACTUAL_ARRIVAL_TIME']-df['ARRIVAL_TIME']))
df = df.withColumn('DEPARTURE_DELAY', F.when((df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME'] )< 0, 0).otherwise(df['ACTUAL_DEPARTURE_TIME']-df['DEPARTURE_TIME']))

#convert seconds in minutes
df = df.withColumn('ARRIVAL_DELAY', F.round(df['ARRIVAL_DELAY']/60))
df = df.withColumn('DEPARTURE_DELAY', F.round(df['DEPARTURE_DELAY']/60))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
%%spark

#define HOUR_ARRIVAL
df = df.withColumn('HOUR_ARRIVAL', F.date_format(F.from_unixtime(df['ARRIVAL_TIME']), 'HH:mm:ss'))
df = df.withColumn('HOUR_DEPARTURE', F.date_format(F.from_unixtime(df['DEPARTURE_TIME']), 'HH:mm:ss'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Filter data based on simplifying assumptions

We only consider journeys at reasonable hours of the day, and on a typical business day.
- `day`: from Monday to Friday
- `hours`: first departure at 6:00, last arrival at 21:00

In [14]:
%%spark

#business day
df = df.filter(df['DAY_OF_WEEK'] <6)
#reasonable hours of the day
df = df.filter(F.hour(F.from_unixtime(df['DEPARTURE_TIME']))>=8)
df = df.filter(F.hour(F.from_unixtime(df['ARRIVAL_TIME']))<=20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
%%spark

df.filter(df.TRIP_ID == '80:06____:17010:000').orderBy(df.TRIP_ID, df.HOUR_DEPARTURE).select(df.TRIP_ID, df.HOUR_DEPARTURE, df.STOP_ID, df.STOP_NAME).distinct().show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------------+-------+------------+
|TRIP_ID            |HOUR_DEPARTURE|STOP_ID|STOP_NAME   |
+-------------------+--------------+-------+------------+
|80:06____:17010:000|08:28:00      |8500090|Basel Bad Bf|
+-------------------+--------------+-------+------------+

### Obtain delay cumulative distribution

In [16]:
%%spark
from pyspark.sql import Window

#compute cumulative distribution of delay over groupBy 'DAY_OF_WEEK', 'PRODUCT_ID', 'HOUR_ARRIVAL' 
cumulative_function_window = Window.partitionBy('DAY_OF_WEEK', 'TRIP_ID', 'HOUR_ARRIVAL', 'STOP_ID', 'STOP_NAME').orderBy('ARRIVAL_DELAY').rangeBetween(Window.unboundedPreceding, 0)
count_window = Window.partitionBy('DAY_OF_WEEK', 'TRIP_ID', 'HOUR_ARRIVAL', 'STOP_ID', 'STOP_NAME')

df2 = df.groupBy(df['DAY_OF_WEEK'], df['TRIP_ID'], df['HOUR_ARRIVAL'], df['STOP_ID'], df['STOP_NAME'], df['ARRIVAL_DELAY']).count()
df2 = df2.withColumn("TOTAL_SUM", F.sum("count").over(count_window))
df2 = df2.withColumn("PARTIAL_SUM", F.sum("count").over(cumulative_function_window))
df2 = df2.withColumn("CUMULATIVE", F.sum("count").over(cumulative_function_window)/F.sum("count").over(count_window))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
%%spark

#print for understanding
df2.filter(df2.CUMULATIVE < 1.0).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------------+------------+-------+------------+-------------+-----+---------+-----------+-------------------+
|DAY_OF_WEEK|        TRIP_ID|HOUR_ARRIVAL|STOP_ID|   STOP_NAME|ARRIVAL_DELAY|count|TOTAL_SUM|PARTIAL_SUM|         CUMULATIVE|
+-----------+---------------+------------+-------+------------+-------------+-----+---------+-----------+-------------------+
|          1|85:11:10157:001|    14:40:00|8501609|        Brig|          0.0|    5|        6|          5| 0.8333333333333334|
|          1| 85:11:1067:001|    12:40:00|8501609|        Brig|          0.0|   34|       77|         34|0.44155844155844154|
|          1| 85:11:1067:001|    12:40:00|8501609|        Brig|          1.0|   23|       77|         57| 0.7402597402597403|
|          1| 85:11:1067:001|    12:40:00|8501609|        Brig|          2.0|    9|       77|         66| 0.8571428571428571|
|          1| 85:11:1067:001|    12:40:00|8501609|        Brig|          3.0|    9|       77|         75|  0.974025974

In [18]:
%%spark

#reorder columns
df2 = df2.select(df2["TRIP_ID"], df2["STOP_ID"], df2["DAY_OF_WEEK"], df2["HOUR_ARRIVAL"], df2["ARRIVAL_DELAY"].alias("MAX_ARRIVAL_DELAY"), df2["CUMULATIVE"])
df2 = df2.orderBy(df2["TRIP_ID"], df2["HOUR_ARRIVAL"], df2["HOUR_ARRIVAL"], df2["MAX_ARRIVAL_DELAY"])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Cannot resolve column name "PRODUCT_ID" among (DAY_OF_WEEK, TRIP_ID, HOUR_ARRIVAL, STOP_ID, STOP_NAME, ARRIVAL_DELAY, count, TOTAL_SUM, PARTIAL_SUM, CUMULATIVE);'
Traceback (most recent call last):
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1161, in __getitem__
    jc = self._jdf.apply(item)
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Cannot resolv

### Compute condifence given delay

In [19]:
%%spark

delay = 1 #minutes

df2.filter(df2['MAX_ARRIVAL_DELAY'] == delay).show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
u'Cannot resolve column name "MAX_ARRIVAL_DELAY" among (DAY_OF_WEEK, TRIP_ID, HOUR_ARRIVAL, STOP_ID, STOP_NAME, ARRIVAL_DELAY, count, TOTAL_SUM, PARTIAL_SUM, CUMULATIVE);'
Traceback (most recent call last):
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/pyspark.zip/pyspark/sql/dataframe.py", line 1161, in __getitem__
    jc = self._jdf.apply(item)
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/hdata/sdf/hadoop/yarn/local/usercache/eric/appcache/application_1652960972356_1310/container_e05_1652960972356_1310_01_000001/pyspark.zip/pyspark/sql/utils.py", line 69, in deco
    raise AnalysisException(s.split(': ', 1)[1], stackTrace)
AnalysisException: u'Cannot